Эта статья посвящена следующему вопросу: как доказать, что некая функция потерь является наилучшей в некой задаче? Интуитивно мне всегда казалось, что теория машинного обучения не дает ответа на этот вопрос, и выяснять это нужно либо экспериментально, либо вводя какую-то метрику схожести на целевой переменной на основе знаний из предметной области.

...

Выбор той или иной функции потерь в задачах машинного обучения может быть либо сделан экспериментально, либо обоснован теоретическими рассуждениями разного рода. Например следующими:

**Легкость оптимизации**

Часто требуется, чтобы функция потерь была дифференцируемой, иначе многие алгоритмы оптимизации просто не будут работать. Оптимизация часто осуществляется градиентным спуском, и потому полезно исследовать *градиенты, которые дает функция потерь*. Пара примеров:

*Пример 1.* Функция потерь MAE дает один и тот же по модулю градиент, независимо от величины ошибки. В функции потерь MSE, напротив, градиент тем меньше, чем меньше величина ошибки. Это свойство часто улучшает сходимость методов оптимизации. Также оно соответствует здравому смыслу: чем больше ошибка, тем сильнее ее нужно "штрафовать". Хотя такие рассуждения следует применять с осторожностью.

*Пример 2.* В задаче бинарной классификации мы можем использовать бинарную кроссэнтропию или MSE. Предположим, что до функции потерь находится сигмоида $\sigma(x)$. Посчитаем производную функции потерь по аргументу сигмоиды для случая, когда верный ответ равен 1. В случае бинарной кроссэнтропии:

$\cfrac{\partial {loss}}{\partial x} = 1 - \sigma(x)$

В случае MSE:

$\cfrac{\partial {loss}}{\partial x} = -2\sigma(x)(1-\sigma(x))^2$

При $x \to -\infty$ (то есть когда модель выдает уверенный неправильный ответ) при бинарной кроссэнтропии производная стремится к 1, а при MSE производная стремится к нулю. Это означает, что при сочетении сигмоиды и MSE уверенные неправильные ответы практически не корректируются (градиент близок к нулю), что может негативно сказаться на качестве обучения. Это аргумент в пользу того, чтобы при использовании сигмоиды выбирать бинарную кроссэнтропию, а не MSE.

Обобщая описанный выше подход, можно даже не выбирать функцию потерь, а выбирать напрямую ее производную, так как только она используется при оптимизации.

**Соображения теории вероятностей**

С позиции теории вероятностей модель машинного обучения можно рассматривать как [статистическую модель]($Статистические модели$) для оценки условной вероятности $P(y|x)$ (то есть зная $x$ оценить вероятности разных значений $y$). Модель представляется как параметризованное семейство гипотез:

$\{P_\theta(y|x) \,|\, \theta \in \Theta\}$

Для поиска оптимальных параметров $\theta$ можно использовать метод максимального правдоподобия, то есть искать такое $\theta$, при котором верноятность наблюдаемых данных (обучающего датасета) максимальна. Поиск максимума правдоподобия равносилен поиску минимума от минус логарифма правдоподобия. Также можно сопоставить разным значениям $\theta$ разные априорные вероятности, то есть добавить в модель регуляризацию. В таком случае ищется апостериорный максимум. Более подробно об этом я писал в [статье о статистических моделях]($Статистические модели$).

В задаче классификации модель напрямую оценивает $P(y|x)$, то есть при заданном $x$ ищет вероятности для каждого $y$. Максимизация правдоподобия в этом случае эквивалентна максимизации логарифма $P(y = y_{true}|x)$, что соответствует формуле для категориальной и бинарной кроссэнтропии. Это обосновывает применение кроссэнтропии как функции потерь в задаче классификации.

В задаче регрессии модель как параметризованное семейство гипотез $P(y|x)$ можно задать как нормальное распределение, мат. ожидание которого расчитывается с помощью параметров $\theta$, а дисперсию можно задать произвольно:

$P_\theta(y|x) = \mathcal{N}(y; f(x, \theta), \sigma)$

Если в данном выражении мы распишем формулу для нормального распределения, то увидим, что максимизация правдоподобия для пары $(x, y)$ из датасета равносильна минимизации MSE между $f(x, \theta)$ и $y$. Изменение параметра $\sigma$ при этом равносильно масштабированию функции потерь или, что то же самое, изменению learning rate. Это обосновывает применение функции потерь MSE в задачах регрессии.

Впрочем, вместо нормального распределения можно задать и любое другое. Выбор именно *нормального* распределения в некоторых случаях может быть неоптимальным.

**Соображения теории информации**

Метод максимального правдоподобия можно рассматривать как минимизацию расхождения Кульбака-Лейблера между [эмпирическим распределением](https://en.wikipedia.org/wiki/Empirical_distribution_function) и задаваемым моделью распределением $P_\theta(y|x)$. Эмпирическое распределение - это дискретное распределение вероятностей, полученное напрямую из обучающих данных. Для пары $(x_i, y_i)$ из обучающих данных эмпирическое распределение задается следующим образом:

$P(y, x_i) = \begin{cases} 1, y = y_i \\ 0, y \neq y_i \end{cases}$

Если мы имеем два распределения $P$ и $Q$, то минимизация по $Q$ расхождения Кульбака-Лейблера между $P$ и $Q$ равносильна минимизации кроссэнтропии между $P$ и $Q$. Кроссэнтропия определяется по следующей формуле для дискретного и непрерывного распределений:

$H(P, Q) = -\sum\limits_{x \in X} P(x)\log Q(x)$

$H(p, q) = -\int\limits_X p(x)\log q(x)\,dx$

В задаче классификации это и есть бинарная/категориальная кроссэнтропия, используемая часто в качестве функции потерь.

В случае регрессии мы задавали плотность вероятности $P_\theta(y|x)$ следующим образом:

$P_\theta(y|x) = \mathcal{N}(y; f(x, \theta), \sigma)$

Плотность вероятности эмпирического распределения задается функцией Дирака. Прямым подсчетом можно увидеть, что кроссэнтропия между эмпирическим распределением и задаваемым моделью распределением равна MSE между $f(x, \theta)$ и $y$.

Таким образом, если обучающая выборка доступна в виде пар $(x, y)$, то есть в виде эмпирического распределения, то максимизация правдоподобия и минимизация кроссэнтропии - одно и то же. Впрочем, понятие кроссэнтропии основано на теории информации, которая первоначально использовалась в теории кодирования сигналов. Сложно сказать насколько правомерным является использование тех же идей в задачах статистического вывода, то есть в совсем другой области.

**Функция потерь как выражение метрики на целевой переменной**

Отличие классификации от регрессии в том, что в задаче классификации все классы считаются одинаково непохожими друг на друга, тогда как в задаче регрессии чем ближе два значения, тем более они схожи друг с другом. Это можно рассматривать как применения гипотезы компактности к целевым данным. Именно это определяет тот факт, что в классификации и регрессии применяются разные форматы выходных данных и разные функции потерь.

Метрика схожести может быть и более сложной, например в случае иерархической классификации мы заранее знаем, что некоторые классы более похожи друг на друга, чем другие. Например, в задаче классификации изображений перепутать породу собаки на изображении является в меньшей степени ошибкой, чем спутать собаку с автомобилем. Мы можем явно указать метрику расстояния между классами и создать нестандартный формат выходных данных и/или нестандартную функцию потерь, учитывающую эту метрику. Тогда наша задача будет иметь черты и классификации, и регрессии.

Таким образом, *априорная метрика схожести на множестве целевых данных определяет оптимальный формат выходных данных и функцию потерь.* Тем самым мы выбираем априорные гипотезы о виде распределения $P(y|x)$. В задаче классификации распределение $P(y|x)$ может иметь какой угодно вид. В задаче регрессии из-за гипотезы компактности можно предположить, что существуют такие параметры $\theta$, что $P(y|x) = \mathcal{N}(y; f(x, \theta), \sigma)$, и решение мы ищем именно в таком виде (вместо $\mathcal{N}$ может быть и другое распределение).

В принципе задачу классификации можно решать и как регрессию, напрямую предсказывая номер класса и округляя его до целого. Это по сути будет означать, что мы применяем к номеру класса гипотезу компактности: классы с похожими номерами более схожи. Если это безосновательно (а как правило так и есть), то и решение получится существенно хуже по метрике качества. И наоборот, задачу регрессии можно решать как задачу классификации, разбив числовую прямую на интервалы и предсказывая вероятность принадлежности объекта каждому интервалу. Тем самым мы избавляемся от гипотезы компактности. Если это безосновательно, то решение также получится существенно хуже. Однако если доступно очень большое количество и разнообразие обучающих данных, то априорные гипотезы перестают играть столь важную роль. В принципе при огромном объеме обучающих данных и высокой выразительной способности модели мы можем решать классификацию как регрессию или наоборот, и получим хорошую точность.